In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from TidalPy.toolbox.conversions import days2rads, rads2days, semi_a2orbital_motion
from TidalPy.rheology.complex_compliance.compliance_models import maxwell, sundberg, andrade
from TidalPy.tides.multilayer.numerical_int.solver import tidal_y_solver

R_Venus = 6050.0e3
M_Venus = 4.867e24
planet_bulk_density = M_Venus / ((4. / 3.) * np.pi * R_Venus**3)
venus_solar_day_freq = days2rads(116.75)
solar_day_forcing_frequency = venus_solar_day_freq

# Build Domain
N = 40
# Start at a radius slightly larger than 0.
radius_array_baseline = np.linspace(10., R_Venus, N)

# Rheology Inputs
alpha_baseline, zeta_baseline, andrade_critical_freq_baseline = 0.3, 1., days2rads(10000.)
rheology_inputs_baseline = (alpha_baseline, zeta_baseline, andrade_critical_freq_baseline)

# Integration parameters
int_rtol_scale = 1.
int_atol_scale = .01
use_numba_integrator = True

# Incompressibility Check
incomp_bulk = 1.0e18

In [ ]:

# Build function inputs
N = 100
radius_array = np.linspace(0.1, R_Venus, N)

data = pd.read_csv('venus_data.csv')
radius_array = data['rad'].to_numpy()
density_array = data['rho'].to_numpy()
pressure_array = data['P'].to_numpy()
temperature_array = data['T'].to_numpy()
velocity_p = data['Vp'].to_numpy()
velocity_s = data['Vs'].to_numpy()
viscosity_array = data['visco'].to_numpy()
gravity_array = data['g'].to_numpy()

# Calculate shear and bulk modulus
shear_array = velocity_s**2 * density_array
bulk_array = velocity_p**2 * density_array - (4. / 3.) * shear_array

# Make any corrections
bulk_array[bulk_array < 0.] = 0.
shear_array[shear_array < 0.] = 0.

# Find the cut off between the inner core (if present), outer core, and mantle -- based on the shear velocity
oc_index = velocity_s == 0.
oc_radii = radius_array[oc_index]
cmb_radius = oc_radii[-1]
icb_radius = oc_radii[0]
mantle_index = radius_array > cmb_radius
ic_index = np.logical_not(np.logical_or(oc_index, mantle_index))

complex_shear = maxwell(venus_solar_day_freq, )

function_inputs = (
'solid_liquid_solid', radius_array, 

)
load_y = tidal_y_solver('solid',
                            radius_array_, complex_shear, bulk_array_, density_array_,
                            gravity_array_, forcing_frequency,
                            is_solid_by_layer, is_static_by_layer, indices_by_layer,
                            order_l=order_l,
                            solve_load_numbers=True, nondimensionalize=True,
                            use_numba_integrator=use_numba_integrator, int_rtol=int_rtol, int_atol=int_atol,
                            planet_bulk_density=planet_bulk_density)